# IMDB 영화리뷰 감성분석 - Conv1D
- Conv1D + Conv1D + Dense
- Conv1D + LSTM + Dense
- Conv1D + Dense + Dense

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [8]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [5]:
num_words = 10000
(X_train, y_train), (_, _) = imdb.load_data(num_words=num_words)

17473536/17464789 [==============================] - 0s 0us/step


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, stratify=y_train, test_size=0.2, random_state=seed
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000,), (5000,), (20000,), (5000,))

In [7]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((20000, 500), (5000, 500))

- Case 1) Conv1D x 2

In [9]:
model1 = Sequential([
                     Embedding(num_words, 100, input_length=max_len),
                     Dropout(0.5),
                     Conv1D(64, 7, activation='relu'),
                     MaxPooling1D(7),
                     Conv1D(64, 5, activation='relu'),
                     MaxPooling1D(5),
                     GlobalMaxPooling1D(),
                     Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 500, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                    

In [12]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])
model1_path = 'best-imdb-conv1d-conv1d.h5'
mc1 = ModelCheckpoint(model1_path, verbose=1, save_best_only=True)
es1 = EarlyStopping(patience=5)

In [13]:
hist1 = model1.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc1,es1]
)

Epoch 1/30
245/250 [============================>.] - ETA: 0s - loss: 0.5606 - accuracy: 0.6746
Epoch 1: val_loss improved from inf to 0.35860, saving model to best-imdb-conv1d-conv1d.h5
250/250 [==============================] - 14s 10ms/step - loss: 0.5564 - accuracy: 0.6783 - val_loss: 0.3586 - val_accuracy: 0.8475
Epoch 2/30
246/250 [============================>.] - ETA: 0s - loss: 0.2872 - accuracy: 0.8827
Epoch 2: val_loss improved from 0.35860 to 0.32364, saving model to best-imdb-conv1d-conv1d.h5
250/250 [==============================] - 2s 8ms/step - loss: 0.2879 - accuracy: 0.8819 - val_loss: 0.3236 - val_accuracy: 0.8587
Epoch 3/30
250/250 [==============================] - ETA: 0s - loss: 0.1937 - accuracy: 0.9262
Epoch 3: val_loss did not improve from 0.32364
250/250 [==============================] - 2s 8ms/step - loss: 0.1937 - accuracy: 0.9262 - val_loss: 0.3492 - val_accuracy: 0.8553
Epoch 4/30
244/250 [============================>.] - ETA: 0s - loss: 0.1318 - accur

In [14]:
best_model1 = load_model(model1_path)
best_model1.evaluate(X_test, y_test)

157/157 [==============================] - 1s 3ms/step - loss: 0.3087 - accuracy: 0.8688


[0.308663934469223, 0.8687999844551086]

- Case 2) Conv1D + LSTM

In [10]:
model2 = Sequential([
                     Embedding(num_words, 100, input_length=max_len),
                     Dropout(0.5),
                     Conv1D(64, 5, activation='relu'),
                     MaxPooling1D(5),
                     LSTM(100),
                     Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                      

In [15]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])
model2_path = 'best-imdb-conv1d-lstm.h5'
mc2 = ModelCheckpoint(model2_path, verbose=1, save_best_only=True)
es2 = EarlyStopping(patience=5)

In [16]:
hist2 = model2.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc2,es2]
)

Epoch 1/30
248/250 [============================>.] - ETA: 0s - loss: 0.4517 - accuracy: 0.7596
Epoch 1: val_loss improved from inf to 0.29695, saving model to best-imdb-conv1d-lstm.h5
250/250 [==============================] - 6s 14ms/step - loss: 0.4503 - accuracy: 0.7608 - val_loss: 0.2969 - val_accuracy: 0.8863
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.2162 - accuracy: 0.9142
Epoch 2: val_loss improved from 0.29695 to 0.25823, saving model to best-imdb-conv1d-lstm.h5
250/250 [==============================] - 3s 13ms/step - loss: 0.2162 - accuracy: 0.9142 - val_loss: 0.2582 - val_accuracy: 0.8975
Epoch 3/30
250/250 [==============================] - ETA: 0s - loss: 0.1498 - accuracy: 0.9433
Epoch 3: val_loss did not improve from 0.25823
250/250 [==============================] - 3s 13ms/step - loss: 0.1498 - accuracy: 0.9433 - val_loss: 0.2799 - val_accuracy: 0.8890
Epoch 4/30
250/250 [==============================] - ETA: 0s - loss: 0.0936 - accuracy

In [17]:
best_model2 = load_model(model2_path)
best_model2.evaluate(X_test, y_test)

157/157 [==============================] - 1s 4ms/step - loss: 0.2718 - accuracy: 0.8904


[0.27176687121391296, 0.8903999924659729]

- Case 3) Conv1D + Dense

In [11]:
model3 = Sequential([
                     Embedding(num_words, 100, input_length=max_len),
                     Dropout(0.5),
                     Conv1D(64, 5, activation='relu'),
                     MaxPooling1D(5),
                     GlobalMaxPooling1D(),
                     Dense(100, activation='relu'),
                     Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_2 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 100)              

In [18]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])
model3_path = 'best-imdb-conv1d-dense.h5'
mc3 = ModelCheckpoint(model3_path, verbose=1, save_best_only=True)
es3 = EarlyStopping(patience=5)

In [19]:
hist3 = model3.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc3,es3]
)

Epoch 1/30
243/250 [============================>.] - ETA: 0s - loss: 0.5828 - accuracy: 0.6576
Epoch 1: val_loss improved from inf to 0.38874, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 3s 9ms/step - loss: 0.5772 - accuracy: 0.6622 - val_loss: 0.3887 - val_accuracy: 0.8275
Epoch 2/30
243/250 [============================>.] - ETA: 0s - loss: 0.3119 - accuracy: 0.8691
Epoch 2: val_loss improved from 0.38874 to 0.29926, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 2s 7ms/step - loss: 0.3117 - accuracy: 0.8698 - val_loss: 0.2993 - val_accuracy: 0.8830
Epoch 3/30
248/250 [============================>.] - ETA: 0s - loss: 0.2078 - accuracy: 0.9206
Epoch 3: val_loss improved from 0.29926 to 0.28892, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 2s 7ms/step - loss: 0.2080 - accuracy: 0.9206 - val_loss: 0.2889 - val_accuracy: 0.8842
Epoch 4/30
247/250 [=====================

In [20]:
best_model3 = load_model(model3_path)
best_model3.evaluate(X_test, y_test)

157/157 [==============================] - 1s 3ms/step - loss: 0.2822 - accuracy: 0.8866


[0.28215670585632324, 0.8866000175476074]